### Konstantina Andronikou

## This notebook is implementing Negation as a Robustness test

In [2]:
from allennlp_models.pretrained import load_predictor

In [3]:
import checklist
import logging 
import csv
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [4]:
from checklist.pred_wrapper import PredictorWrapper

In [5]:
#removing the warnings
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True
srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("John has pets.")
output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'has',
   'description': '[ARG0: John] [V: has] [ARG1: pets] .',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O']}],
 'words': ['John', 'has', 'pets', '.']}

In [6]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [7]:
d = ["John has pets"]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'has',
     'description': '[ARG0: John] [V: has] [ARG1: pets]',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1']}],
   'words': ['John', 'has', 'pets']}],
 array([1.]))

In [8]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [9]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [10]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [11]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [12]:
editor = Editor()

# create examples
t = editor.template('{first_name} has pets.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Andrew'}, {'first_name': 'Edith'}, {'first_name': 'Barbara'}, {'first_name': 'Lauren'}, {'first_name': 'Lawrence'}, {'first_name': 'Lauren'}, {'first_name': 'Betty'}, {'first_name': 'Annie'}, {'first_name': 'Betty'}, {'first_name': 'Frank'}, {'first_name': 'James'}, {'first_name': 'Cynthia'}, {'first_name': 'Jason'}, {'first_name': 'Keith'}, {'first_name': 'Victoria'}, {'first_name': 'Joan'}, {'first_name': 'Judith'}, {'first_name': 'Katherine'}, {'first_name': 'Diana'}, {'first_name': 'Sally'}, {'first_name': 'George'}, {'first_name': 'Billy'}, {'first_name': 'Gary'}, {'first_name': 'Evelyn'}, {'first_name': 'Alison'}, {'first_name': 'Peter'}, {'first_name': 'Jonathan'}, {'first_name': 'Ashley'}, {'first_name': 'Margaret'}, {'first_name': 'Eleanor'}, {'first_name': 'Donna'}, {'first_name': 'William'}, {'first_name': 'Harry'}, {'first_name': 'Harry'}, {'first_name': 'Jane'}, {'first_name': 'Frank'}, {'first_name': 'Eleanor'}, {'first_name': 'Sandra'}, {'first_name

In [14]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/without_negation_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)
['Andrew', 'has', 'pets', '.'] [ True]
['Edith', 'has', 'pets', '.'] [ True]
['Barbara', 'has', 'pets', '.'] [ True]
['Lauren', 'has', 'pets', '.'] [ True]
['Lawrence', 'has', 'pets', '.'] [ True]
['Lauren', 'has', 'pets', '.'] [ True]
['Betty', 'has', 'pets', '.'] [ True]
['Annie', 'has', 'pets', '.'] [ True]
['Betty', 'has', 'pets', '.'] [ True]
['Frank', 'has', 'pets', '.'] [ True]
['James', 'has', 'pets', '.'] [ True]
['Cynthia', 'has', 'pets', '.'] [ True]
['Jason', 'has', 'pets', '.'] [ True]
['Keith', 'has', 'pets', '.'] [ True]
['Victoria', 'has', 'pets', '.'] [ True]
['Joan', 'has', 'pets', '.'] [ True]
['Judith', 'has', 'pets', '.'] [ True]
['Katherine', 'has', 'pets', '.'] [ True]
['Diana', 'has', 'pets', '.'] [ True]
['Sally', 'has', 'pets', '.'] [ True]
['George', 'has', 'pets', '.'] [ True]
['Billy', 'has', 'pets', '.'] [ True]
['Gary', 'has', 'pets', '.'] [ True]
['Evelyn', 'has', 'pets', '.'] [ Tru

In [15]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Output/false_sentences_without_negation.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Addinig Negation 

In [16]:
editor = Editor()

# create examples
t = editor.template('{first_name} has no pets.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Ben'}, {'first_name': 'Nick'}, {'first_name': 'Julie'}, {'first_name': 'Emma'}, {'first_name': 'Julia'}, {'first_name': 'Edward'}, {'first_name': 'Kelly'}, {'first_name': 'Sophie'}, {'first_name': 'Ryan'}, {'first_name': 'Walter'}, {'first_name': 'Amanda'}, {'first_name': 'Tony'}, {'first_name': 'Johnny'}, {'first_name': 'Diana'}, {'first_name': 'Betty'}, {'first_name': 'Don'}, {'first_name': 'Florence'}, {'first_name': 'Jimmy'}, {'first_name': 'Roger'}, {'first_name': 'Rose'}, {'first_name': 'Walter'}, {'first_name': 'Nick'}, {'first_name': 'Nick'}, {'first_name': 'Sarah'}, {'first_name': 'Julie'}, {'first_name': 'Andrew'}, {'first_name': 'Jason'}, {'first_name': 'Alison'}, {'first_name': 'Steven'}, {'first_name': 'Janet'}, {'first_name': 'Lucy'}, {'first_name': 'Diane'}, {'first_name': 'Alexandra'}, {'first_name': 'Tom'}, {'first_name': 'Jason'}, {'first_name': 'Tony'}, {'first_name': 'Martin'}, {'first_name': 'Roger'}, {'first_name': 'Jill'}, {'first_name': 'Be

In [17]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/with_negation.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)
['Ben', 'has', 'no', 'pets', '.'] [ True]
['Nick', 'has', 'no', 'pets', '.'] [ True]
['Julie', 'has', 'no', 'pets', '.'] [ True]
['Emma', 'has', 'no', 'pets', '.'] [ True]
['Julia', 'has', 'no', 'pets', '.'] [ True]
['Edward', 'has', 'no', 'pets', '.'] [ True]
['Kelly', 'has', 'no', 'pets', '.'] [ True]
['Sophie', 'has', 'no', 'pets', '.'] [ True]
['Ryan', 'has', 'no', 'pets', '.'] [ True]
['Walter', 'has', 'no', 'pets', '.'] [ True]
['Amanda', 'has', 'no', 'pets', '.'] [ True]
['Tony', 'has', 'no', 'pets', '.'] [ True]
['Johnny', 'has', 'no', 'pets', '.'] [ True]
['Diana', 'has', 'no', 'pets', '.'] [ True]
['Betty', 'has', 'no', 'pets', '.'] [ True]
['Don', 'has', 'no', 'pets', '.'] [ True]
['Florence', 'has', 'no', 'pets', '.'] [ True]
['Jimmy', 'has', 'no', 'pets', '.'] [ True]
['Roger', 'has', 'no', 'pets', '.'] [ True]
['Rose', 'has', 'no', 'pets', '.'] [ True]
['Walter', 'has', 'no', 'pets', '.'] [ True]
['N

In [18]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Output/false_sentences_with_negation.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 